## Image Augmentation Framework for the Object detection model

### Necessary packages

In [2]:
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np
import pandas as pd
import os
import cv2
from matplotlib import pyplot as plt
from itertools import combinations
from tqdm import tqdm
import gc
gc.enable()
%matplotlib inline

In [3]:
%cd train/images/

/home/camivasz/A-nowcasting-model-for-Medellin-city/train/images


### Images reading

In [4]:
test = []
for i in os.listdir("test"):
    if i[-4:] == ".jpg": test.append(i)
train = []
for i in os.listdir("train"):
    if i[-4:] == ".jpg": train.append(i)
tt = {}
tr = {}
for i in range(len(test)):
    tt[test[i]] = (cv2.imread("test/"+test[i]))

for i in range(len(train)):
    tr[train[i]] = (cv2.imread("train/"+train[i]))

## Augmentations
We are going to apply some transformations to all the images, such as flipping, blur, noise and affine transform.

In [5]:
test_labels = pd.read_csv("test_labels.csv",index_col = 0)
train_labels = pd.read_csv("train_labels.csv", index_col = 0)

In [6]:
!mkdir augmented

In [7]:
# Rotación
seq1 = iaa.Sequential([iaa.Affine(rotate=np.random.choice([10, 15, 20, 25]), fit_output = True)])

# Sharpen
seq11 = iaa.Sequential([iaa.Sharpen(alpha=1.0)])

# Corte
seq2 = iaa.Sequential([iaa.CropAndPad(percent=(-0.3, 0.3))])

# Add element wise
seq22 = iaa.Sequential([iaa.AddElementwise((-40, 40))])

# Dropout
seq3 = iaa.Sequential([iaa.Dropout(p=(0, 0.2))])

# Gaussian Blur
seq33 = iaa.Sequential([iaa.GaussianBlur(sigma=(0.0, 3.0))])

# Motion Blur
seq4 = iaa.Sequential([iaa.MotionBlur(angle = 40, k = 35)])

# Emboss
seq44 = iaa.Sequential([iaa.Emboss(alpha=(0.0, 1.0), strength=(0.5, 1.5))])

# color change
seq5 = iaa.Sequential([iaa.WithColorspace(
    to_colorspace="HSV",
    from_colorspace="RGB",
    children=iaa.WithChannels(0, iaa.Add((10, 50)))
)])

# Piecewise affine
seq55 = iaa.Sequential([iaa.PiecewiseAffine(scale=(0.01, 0.05))])

# increase redness
seq6 = iaa.Sequential([iaa.WithChannels(0, iaa.Add((10, 100)))])

# Coarse dropout
seq66 = iaa.Sequential([iaa.CoarseDropout((0.0, 0.05), size_percent=(0.02, 0.25))])

# Flipr
seq7 = iaa.Sequential([iaa.Fliplr(0.5)])

# Coarse dropout with color
seq77 = iaa.Sequential([iaa.CoarseDropout(0.02, size_percent=0.15, per_channel=0.5)])

# colorspace change
seq8 = iaa.Sequential([iaa.Sequential([
    iaa.ChangeColorspace(from_colorspace="RGB", to_colorspace="HSV"),
    iaa.WithChannels(0, iaa.Add((50, 100))),
    iaa.ChangeColorspace(from_colorspace="HSV", to_colorspace="RGB")
])])

# contrast normalization
seq88 = iaa.Sequential([iaa.ContrastNormalization((0.5, 1.5))])

# grayscale
seq9 = iaa.Sequential([iaa.Grayscale(alpha=(0.0, 1.0))])

# Affine moving
seq99 = iaa.Sequential([iaa.Affine(translate_px={"x": (-20, 20), "y": (-20, 20)})])

augmentations = [
    seq1,
    seq2,
    seq3,
    seq4,
    seq5,
    seq6,
    seq7,
    seq8,
    seq9,
    seq11,
    seq22,
    seq33,
    seq44,
    seq55,
    seq66,
    seq77,
    seq88,
    seq99
]

### Mixing them up
We will mix the different augmentations with each other.

In [18]:
aa = []
for j in range(1, 5):
    for i in combinations(range(len(augmentations)), j):
        aa.append(i)

In [20]:
black_list = set()

### Let the augmentation begin
The following function performs a set of augmentations 'aug', in an image 'img', with index 'aa' (this is used to name the new image). The parameter 'tes' indicates wether the image is in the test set or not. It also augmentates the bounding boxes, saves the image and returns the DataFrame associated with the labels in the bounding boxes.

In [22]:
def performAugmentation(augs, aa, img, tes = True):
    try:
        if tes:
            r = test_labels.loc[img,:].copy()
            if type(r) != type(test_labels):
                re = pd.DataFrame(r).transpose() 
            else:
                re = r
            re = re.rename(index = {img:'augmented/'+str(aa)+img})
            image = cv2.imread('test/'+img)
        else:
            r = train_labels.loc[img,:].copy()
            if type(r) != type(test_labels):
                re = pd.DataFrame(r).transpose() 
            else:
                re = r
            re = re.rename(index = {img:'augmented/'+str(aa)+img})
            image = cv2.imread('train/'+img)
            
        a = re.shape[0]
        bb = []
        img_aug = image
        for i in range(a):
            bb.append(ia.BoundingBox(x1 = list(re['xmin'])[i], 
                                     x2 = list(re['xmax'])[i],
                                     y1 = list(re['ymin'])[i],
                                     y2 = list(re['ymax'])[i]))
        # agumentation
        bb = ia.BoundingBoxesOnImage(bb, shape = tt[test[1]].shape)
        bbs_aug = bb
        for i in augs:            
            image_aug = augmentations[i].augment_images([image])[0]
            bbs_aug = augmentations[i].augment_bounding_boxes([bb])[0]
            image = img_aug
            bb = bbs_aug
        
        cv2.imwrite('augmented/'+str(aa)+img, image_aug)

        xmin = []
        xmax = []
        ymin = []
        ymax = []
        for i in bbs_aug.bounding_boxes:
            xmin.append(i.x1)
            xmax.append(i.x2)
            ymin.append(i.y1)
            ymax.append(i.y2)
        re['xmin'] = xmin
        re['xmax'] = xmax
        re['ymin'] = ymin
        re['ymax'] = ymax

        return re
    except:
           black_list.add(img)

Now, with the read images in the train and test directories, we perform the augmentations.

In [23]:
for i in tqdm(range(len(aa))):
    tes = True
    for j in test:
        re = performAugmentation(aa[i], i, j, tes)
        test_labels = pd.concat([test_labels, re])
    tes = False
    for j in train: 
        re = performAugmentation(aa[i], i, j, tes)
        train_labels = pd.concat([train_labels, re])

  7%|▋         | 294/4047 [7:59:45<224:16:28, 215.13s/it]

KeyboardInterrupt: 

### Save your work!
We save a .csv containing all of the images with their labels. 

In [ ]:
test_labels.to_csv('test_labels.csv')
train_labels.to_csv('train_labels.csv')